In [ ]:
# install and imports required package 

In [ ]:
!pip install pyenchant

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 KB 2.5 MB/s eta 0:00:00


In [ ]:
import enchant
 
# dictionary of en_US
d = enchant.Dict("en_US")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import nltk
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict 
import numpy as np
from statistics import variance
from sklearn import preprocessing

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/nlp/Clickbit_dataset/clickbait17-validation-170630/feature/csv/para_and_mean_variance.csv")

In [ ]:
data.head()

,Unnamed: 0,id,paragraph,postText mean,postText variance,title mean,title variance
0,0,858462320779026433,Thousands of modern slavery victims havenot co...,0.643028,0.016918,0.678807,0.020574
1,1,858421020331560960,President Donald Trump has appointed thepro-li...,0.259717,0.070382,0.428651,0.059395
2,2,858368123753435136,When theWhite House correspondents’ dinner is ...,0.619952,0.026573,0.631976,0.024935
3,3,858323428260139008,Adorable is probably an understatement. This a...,0.409056,0.056876,0.361099,0.105833
4,4,858283602626347008,One of Tokyo's major subways systems says it s...,0.596689,0.029145,0.596689,0.029145


In [ ]:
paragraph =  data["paragraph"]

In [ ]:
paragraph

0        Thousands of modern slavery victims havenot co...
1        President Donald Trump has appointed thepro-li...
2        When theWhite House correspondents’ dinner is ...
3        Adorable is probably an understatement. This a...
4        One of Tokyo's major subways systems says it s...
                               ...                        
19533    Watch Live:Joe Biden Honored on Senate FloorBr...
19534    On November 30 Politico reported that Eric Tru...
19535    The Forest High School on Sydney's northern be...
19536    NFL coaches have a lot of information to remem...
19537    Seattle Seahawks cornerback Richard Sherman ma...
Name: paragraph, Length: 19538, dtype: object

In [ ]:
import re
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

for i in range(len(paragraph)):
  t = remove_emoji(paragraph[i])
  paragraph[i] = t

<ipython-input-21-8d920fa84e15>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paragraph[i] = t


In [ ]:
for i in range(len(paragraph)):
  t = paragraph[i].translate(str.maketrans('', '', string.punctuation))
  paragraph[i] = t

<ipython-input-22-16a037f4d97f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paragraph[i] = t


In [ ]:
# finding ratio of formal word and informal word of target paragraph

In [ ]:
def ratio(para):
  l = []
  for item in para:
    cnt_f ,cnt_inf = 0, 0
    str_l = nltk.word_tokenize(item)
    for word in str_l:
      if(d.check(word)==True):
        cnt_f=cnt_f+1
      else:
        cnt_inf = cnt_inf+1
    if(cnt_f==0 or cnt_inf==0):
      l.append(0)
    else:
      l.append(cnt_f/cnt_inf)
  return l

In [ ]:
l = ratio(paragraph)

In [ ]:
print(l[0])

In [ ]:
data_=pd.DataFrame()
data_["id"] = data["id"]
data_["Paragraph Ratio of formal and informal word"] = pd.DataFrame(l)

In [ ]:
data_.shape

(19538, 2)

In [ ]:
data.shape

(19538, 8)

In [ ]:
data_.head()

,id,Paragraph Ratio of formal and informal word
0,858462320779026433,8.821138
1,858421020331560960,5.553571
2,858368123753435136,6.739437
3,858323428260139008,5.000000
4,858283602626347008,10.871795


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/nlp/Clickbit_dataset/clickbait17-validation-170630/feature/csv/common_count_sentiment.csv")

In [ ]:
df.head()

,id,freq of word in para,Match keywords paragraph word,match title and paragraph word,match postText and paragraph word,senti_score,senti_score_absolute,entroy,lexical Diversity,Paragraph Ratio of formal and informal word
0,858462320779026433,"['victims', 'modern', 'slavery', 'support', 'r...",2,6,5,-0.125,0.875,8.342841,0.41,8.821138
1,858421020331560960,"['president', 'prolife', 'yoest', 'trump', 'pa...",13,3,0,0.750,0.750,7.193521,0.55,5.553571
2,858368123753435136,"['trump', 'said', 'roast', 'one', 'new', 'york...",10,7,7,0.000,0.750,8.465862,0.47,6.739437
3,858323428260139008,"['part', 'adorable', 'husky', 'maru', 'also', ...",4,3,0,0.875,0.875,5.538945,0.74,5.000000
4,858283602626347008,"['north', 'missile', 'said', 'says', 'council'...",2,3,3,0.000,0.000,7.505263,0.53,10.871795


In [ ]:
new_fea = pd.merge(df,data_, on ='id', how = 'inner')

In [ ]:
new_fea.head()

,id,freq of word in para,Match keywords paragraph word,match title and paragraph word,match postText and paragraph word,senti_score,senti_score_absolute,entroy,lexical Diversity,Paragraph Ratio of formal and informal word
0,858462320779026433,"['victims', 'modern', 'slavery', 'support', 'r...",2,6,5,-0.125,0.875,8.342841,0.41,8.821138
1,858421020331560960,"['president', 'prolife', 'yoest', 'trump', 'pa...",13,3,0,0.750,0.750,7.193521,0.55,5.553571
2,858368123753435136,"['trump', 'said', 'roast', 'one', 'new', 'york...",10,7,7,0.000,0.750,8.465862,0.47,6.739437
3,858323428260139008,"['part', 'adorable', 'husky', 'maru', 'also', ...",4,3,0,0.875,0.875,5.538945,0.74,5.000000
4,858283602626347008,"['north', 'missile', 'said', 'says', 'council'...",2,3,3,0.000,0.000,7.505263,0.53,10.871795


In [ ]:
new_fea.shape

(19538, 10)

In [ ]:
# Saving the extracted features with existing features 

In [ ]:
new_fea.to_csv("/content/drive/MyDrive/nlp/Clickbit_dataset/clickbait17-validation-170630/feature/csv/common_count_sentiment.csv", index = False)

In [ ]:
label = pd.read_csv('/content/drive/MyDrive/nlp/Clickbit_dataset/clickbait17-validation-170630/Dataset/truths.csv')

In [ ]:
label.head()

,id,truthJudgments,truthMean,truthMedian,truthMode,truthClass,label
0,858462320779026433,"[0.3333333333, 0.0, 0.3333333333, 0.0, 0.0]",0.133333,0.000000,0.000000,no-clickbait,0
1,858421020331560960,"[1.0, 1.0, 1.0, 1.0, 1.0]",1.000000,1.000000,1.000000,clickbait,1
2,858368123753435136,"[0.3333333333, 1.0, 0.3333333333, 0.0, 0.66666...",0.466667,0.333333,0.333333,no-clickbait,0
3,858323428260139008,"[1.0, 0.6666666666, 1.0, 1.0, 1.0]",0.933333,1.000000,1.000000,clickbait,1
4,858283602626347008,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0.000000,0.000000,no-clickbait,0


In [ ]:
da=pd.DataFrame()
da["id"] = df["id"]
da["label"] = label["label"]

In [ ]:
new_fea1 = pd.merge(df,da, on ='id', how = 'inner')